# This is a title!

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import country_converter as coco

Our main topic is about hotel booking marktet.

The reason for choosing the hotel booking market as topic, is because  

-Perspectives

<b>Dataset and preprocessing</b>

Our first dataset is about hotel bookings. This dataset contains data of over 100.000 bookings. It contains data about when the hotel was booked, how long the stay was, the country of the person booking the hotel and much more. In the code below we combined this hotel booking dataset with the countries of the world dataset. This dataset contains information about every country. It tracks data like population, climate and data about the prosperity of the country like GDP and literacy. We aggregated the hotel booking dataset based on country using the mean function. The countries in this dataset are the countries of the person booking the hotel and not the country of the hotel. We changed the country codes in the hotel booking dataset to full country names using the coco package. We did this so we can merge these two datasets based on the country names. We did not have to preproces the countries of the world dataset. 

We also combined the hotel booking dataset with the Air traffic passengers dataset. This dataset has data about Air traffic and it tracks data like operating airline, terminal and passenger count. Both the hotel booking dataset and the Air traffic dataset where aggregated based on the month using the mean function. This allowed us to merge the two datasets based on month. 

These three datasets were found on kaggle. Here are the links: <br>
Hotel bookings - https://www.kaggle.com/datasets/mojtaba142/hotel-booking <br>
Air traffic passengers - https://www.kaggle.com/datasets/thedevastator/airlines-traffic-passenger-statistics <br>
Countries of the world - https://www.kaggle.com/datasets/darshanprabhu09/countries-economy-gdp-and-everything <br>

the final dataset is Average tourist expenditure per month dataset. This dataset contains inforamtion about how much tourist spend on average in each month. It contains inforamtion about how much tourists spend inside and outisde their accomodation, their total spendings and their daily spendings. We changed the month column (which was in greek notation) to two columns, one for regular month names and one for the year. We then aggregated this dataset on the month column using the mean function and then merged it with the hotel booking dataset we aggregated based on month. <br>

The original dataset can found on https://data.europa.eu/data/datasets/fd360c2d-aa17-4a04-909e-6357e300cc96?locale=en 



Code to get the data from the files.

In [3]:
hotel_data = pd.read_table("hotel_booking.csv", delimiter=";")
country_data = pd.read_table("countries-of-the-world.csv", delimiter=",")
air_data = pd.read_table("Air_Traffic_Passenger_Statistics.csv", delimiter=";")

Preprocessing Perspective 1

In [4]:
# group and aggregate the hotel booking dataset based on country using the mean function
aggregate_hotel_country = hotel_data.groupby("country").aggregate({
    'is_canceled': 'mean',
    'lead_time': 'mean',
    'is_repeated_guest': 'mean',
    'previous_cancellations': 'mean',
    'previous_bookings_not_canceled': 'mean',
    'booking_changes': 'mean',
    'stays_in_weekend_nights': 'mean',
    'stays_in_week_nights': 'mean',
    'adults': 'mean',
    'children': 'mean',
    'babies': 'mean',
    'adr': 'mean',
    'required_car_parking_spaces': 'mean',
    'required_car_parking_spaces': 'mean'
})

# change country codes to country names
for i, rows in aggregate_hotel_country.iterrows():
    if not type(i) == float:
        name = coco.convert(i, to='name_short')
        aggregate_hotel_country.rename(index={i : name}, inplace=True)

# merge hotel booking data with country data
concat_country = aggregate_hotel_country.merge(country_data, left_on='country', right_on='Country')

TMP not found in ISO3


Preprocessing Perspective 2

In [5]:
# group and aggregate the hotel booking dataset based on month using the mean function
aggregate_hotel_month = hotel_data.groupby("arrival_date_month").aggregate({
    'is_canceled': 'mean',
    'lead_time': 'mean',
    'is_repeated_guest': 'mean',
    'previous_cancellations': 'mean',
    'previous_bookings_not_canceled': 'mean',
    'booking_changes': 'mean',
    'stays_in_weekend_nights': 'mean',
    'stays_in_week_nights': 'mean',
    'adults': 'mean',
    'children': 'mean',
    'babies': 'mean',
    'adr': 'mean',
    'required_car_parking_spaces': 'mean',
    'required_car_parking_spaces': 'mean'
})

# group and aggregate the air traffic dataset based on month using the mean function
aggregate_air_month = air_data.groupby("Month").aggregate({
    'Passenger Count': 'mean',
    'Adjusted Passenger Count': 'mean'
})

# merge hotel booking data with air traffic data
concat_air = aggregate_hotel_month.merge(aggregate_air_month, left_on='arrival_date_month', right_on='Month')

Visualizations perspective 1

In [15]:
# divide the statistics in equal bins using qcut
qcut_series_lead, qcut_intervals_lead = pd.qcut(concat_country['lead_time'], q=3, retbins=True, labels=['Low', 'Medium', 'High'])
qcut_series_gdp, qcut_intervals_gdp = pd.qcut(concat_country['GDP($percapita)'], q=3, retbins=True, labels=['Low', 'Medium', 'High'])
qcut_series_lit, qcut_intervals_lit = pd.qcut(concat_country['Literacy(%)'], q=3, retbins=True, labels=['Low', 'Medium', 'High'])
qcut_series_phones, qcut_intervals_phones = pd.qcut(concat_country['Phones(per1000)'], q=3, retbins=True, labels=['Low', 'Medium', 'High'], duplicates='drop')
qcut_series_ser, qcut_intervals_ser = pd.qcut(concat_country['Service'], q=3, retbins=True, labels=['Low', 'Medium', 'High'])

# the dimensions/statistics of the graph
lead_dim = go.parcats.Dimension(
    values=qcut_series_lead,
    label="Lead Time",
    categoryorder='array',
    categoryarray=['Low', 'Medium', 'High']
)
gdp_dim = go.parcats.Dimension(
    values=qcut_series_gdp,
    categoryorder='array',
    categoryarray=['Low', 'Medium', 'High'],
    label="GDP"
)
lit_dim = go.parcats.Dimension(
    values=qcut_series_lit,
    categoryorder='array',
    categoryarray=['Low', 'Medium', 'High'],
    label="Literacy"
)
phones_dim = go.parcats.Dimension(
    values=qcut_series_phones,
    categoryorder='array',
    categoryarray=['Low', 'Medium', 'High'],
    label="Phones"
)
ser_dim = go.parcats.Dimension(
    values=qcut_series_ser,
    categoryorder='array',
    categoryarray=['Low', 'Medium', 'High'],
    label="Service"
)

#color coding the low, medium and high values
color = qcut_series_lead.astype('category').cat.codes
colorscale = ['#b0c4de', '#3cb371', '#e9967a']

# create figure with dimensions and color
fig = go.Figure(go.Parcats(
    dimensions=[lit_dim, gdp_dim, lead_dim, phones_dim, ser_dim], 
    line={'color': color, 'colorscale': colorscale}
), layout=go.Layout(title='Analysis of lead time and prosperity measures', height=600))

fig.show()

This visualizations shows that lead time has a correlation to four prosperity measures (Literacy, GDP, Phones, Service). The data is split into low, medium and high to show that something with a low lead time has a high chance of scoring low on the other variables. This is also the case for medium and high. Lead time is the time between reservation and arrival date

In [16]:
# scatter plot using lead time and infant mortality, added trendline, color based on region, hover for country name
fig = px.scatter(
        concat_country, 
        x="lead_time", y="Infantmortality(per1000births)", 
        trendline="ols", trendline_scope="overall", trendline_color_override="black", 
        title='Analysis of lead time and infant mortality rate',
        color='Region', hover_data=['Country']
    )

fig.show()

This visualization shows that there is a correlation between infant mortality and lead time. The x axis shows the average lead time of people in a country. The y axis shows the infant mortality per 1000 births of the countries. The countries are colored based on their respective regions. The trendline (black line) shows that having a higher lead time correlates to having a lower infant mortality rate. Details of the country are shown when hovering over a dot.

Visualizations perspective 2

In [19]:
# visualization of passenger count vs stay in nights
# month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dec']

# passenger count bar
trace_pass = go.Bar(
    x = month_names,
    y = concat_air['Passenger Count'],
    name='Passenger Count'
)

# stays in week nights line
trace_week = go.Scatter(
    x = month_names,
    y = concat_air['stays_in_week_nights'],
    name='Stays in week nights', line=dict(width=4), marker=dict(size=12)
)

# stays in weekend nights line
trace_end = go.Scatter(
    x = month_names,
    y = concat_air['stays_in_weekend_nights'],
    name='Stays in weekend nights', line=dict(width=4), marker=dict(size=12, color='orange')
)

# second y, right side
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace_pass, secondary_y=False)
fig.add_trace(trace_week, secondary_y=True)
fig.add_trace(trace_end, secondary_y=True)

# add title
fig.update_layout(
    title_text="<b>Analysis of airline passenger count and length of hotel stay</b>"
)

# x-axis title
fig.update_xaxes(title_text="<b>Month</b>")

# y-axes titles
fig.update_yaxes(title_text="<b>Passenger Count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Hotel Booking stay nights</b>", secondary_y=True)

fig.show()

The x axis of this visualization shows the months and the x axis shows the average air traffic passenger count on the left side (blue bars) and on the right side it shows the amount of nights in the stay for week (red line) and weekend (green line). We see that all three variables go up and down in roughly the same months. We can see that months with a higher passenger count also have longer stays. 

In [14]:
# visualization of passenger count vs ADR
# month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dec']

# passenger count bar
trace_pass = go.Bar(
    x = month_names,
    y = concat_air['Passenger Count'],
    name='Passenger Count'
)

# adr line
trace_adr = go.Scatter(
    x = month_names,
    y = concat_air['adr'],
    name='Average daily rate (ADR)', line=dict(width=4), marker=dict(size=12)
)

# second y, right side
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace_pass, secondary_y=False)
fig.add_trace(trace_adr, secondary_y=True)

# add title
fig.update_layout(
    title_text="<b>Analysis of airline passenger count and ADR</b>"
)

# x-axis title
fig.update_xaxes(title_text="<b>Month</b>")

# y-axes titles
fig.update_yaxes(title_text="<b>Passenger count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>ADR</b>", secondary_y=True)

fig.show()

This visualization shows that months with a higher passenger count also have a higher ADR. ADR stands for average daily rate and this measures how much people spend on their booking per day. So we can conclude that hotel bookings are more expensive in months were the air traffic passenger count is higher. The x axis shows the months. The left y axis shows the passenger count and the right y axis shows the ADR.